In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import column_or_1d
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
import re
import time as t

#data = pd.read_csv('training_set_features (1).csv', 'training_set_labels (1).csv', 'test_set_features (1).csv')
notebook_path = os.path.abspath("") + '/'

data = pd.read_csv(notebook_path + 'training_set_features (1).csv')
labels = pd.read_csv(notebook_path + 'training_set_labels (1).csv')
test = pd.read_csv(notebook_path + 'test_set_features (1).csv')
feature_list = []

class CustomOrderLabelEncoder(LabelEncoder):
  def fit(self, y):
    y = column_or_1d(y, warn=True)
    self.classes_ = pd.Series(y).unique()
    return self

data.count()

data.values

for (columnName, columnData) in data.items():
    print('Column Name : ', columnName, columnData.dtype)
    print('Unique Contents : ', columnData.unique(), "\n")

le_age_group = CustomOrderLabelEncoder()

le_age_group.fit(['18 - 34 Years', '35 - 44 Years', '45 - 54 Years', '55 - 64 Years', '65+ Years'])

 

data['enc_age_group'] = le_age_group.transform(data['age_group'])
if 'enc_age_group' not in feature_list:
    feature_list.append('enc_age_group')

 

data.loc[1:10, ['enc_age_group', 'age_group']]

le_education = CustomOrderLabelEncoder()

le_education.fit(['< 12 Years', '12 Years', 'Some College', 'College Graduate'])

 

data['enc_education'] = pd.Series(

    le_education.transform(data['education'][data['education'].notnull()]),

    index=data['education'][data['education'].notnull()].index

)
if 'enc_education' not in feature_list:
    feature_list.append('enc_education')

 

data.loc[1:10, ['enc_education', 'education']]

le_income_poverty = CustomOrderLabelEncoder()

le_income_poverty.fit(['Below Poverty', '<= $75,000, Above Poverty', '> $75,000'])

 

data['enc_income_poverty'] = pd.Series(
    
    le_income_poverty.transform(data['income_poverty'][data['income_poverty'].notnull()]),
    
    index=data['income_poverty'][data['income_poverty'].notnull()].index
               
)
if 'enc_income_poverty' not in feature_list:
    feature_list.append('enc_income_poverty')
 

data.loc[1:10, ['enc_income_poverty', 'income_poverty']]

# define One_Hot(prefix, x, y, z= 'blah'):

data = data.drop(columns=['race_White', 'race_Black', 'race_Other or Multiple', 'race_Hispanic'], errors='ignore')

data = pd.concat([data, pd.get_dummies(data.race, prefix='race')], axis=1)

 

data.loc[4107:4111, ['race', 'race_White', 'race_Black', 'race_Other or Multiple', 'race_Hispanic']]
column_list = list(data)
pattern = re.compile(r'^race_(?!.*_nan$).*')
column_list =  list(filter(pattern.match, column_list))
for x in column_list: 
    if x not in feature_list: feature_list.append(x)
print (column_list) 

le_sex = CustomOrderLabelEncoder()

le_sex.fit(['Female', 'Male'])

 

data['enc_sex'] = pd.Series(
    
    le_sex.transform(data['sex'][data['sex'].notnull()]),
    
    index=data['sex'][data['sex'].notnull()].index
               
)
if 'enc_sex' not in feature_list:
    feature_list.append('enc_sex')
 

data.loc[1:10, ['enc_sex', 'sex']]

le_marital_status = CustomOrderLabelEncoder()

le_marital_status.fit(['Not Married', 'Married'])

 

data['enc_marital_status'] = pd.Series(
    
    le_marital_status.transform(data['marital_status'][data['marital_status'].notnull()]),
    
    index=data['marital_status'][data['marital_status'].notnull()].index
               
)
if 'enc_marital_status' not in feature_list:
    feature_list.append('enc_marital_status')
 

data.loc[1:10, ['enc_marital_status', 'marital_status']]

le_rent_or_own = CustomOrderLabelEncoder()

le_rent_or_own.fit(['Own', 'Rent'])

 

data['enc_rent_or_own'] = pd.Series(
    
    le_rent_or_own.transform(data['rent_or_own'][data['rent_or_own'].notnull()]),
    
    index=data['rent_or_own'][data['rent_or_own'].notnull()].index
               
)
if 'enc_rent_or_own' not in feature_list:
    feature_list.append('enc_rent_or_own')
 

data.loc[1:10, ['enc_rent_or_own', 'rent_or_own']]

le_employment_status = CustomOrderLabelEncoder()

le_employment_status.fit(['Not in Labor Force', 'Employed', 'Unemployed'])

 

data['enc_employment_status'] = pd.Series(
    
    le_employment_status.transform(data['employment_status'][data['employment_status'].notnull()]),
    
    index=data['employment_status'][data['employment_status'].notnull()].index
               
)
if 'enc_employment_staus' not in feature_list:
    feature_list.append('enc_employment_status')
 

data.loc[1:10, ['enc_employment_status', 'employment_status']]

data = data.drop(columns=['region_oxchjgsf', 'region_bhuqouqj', 'region_qufhixun', 'region_lrircsnp', 'region_atmpeygn', 'region_lzgpxyit',
 'region_fpwskwrf', 'region_mlyzmhmf', 'region_dqpwygqj', 'region_kbazzjca'], errors='ignore')

data = pd.concat([data, pd.get_dummies(data.hhs_geo_region, prefix='region')], axis=1)

 

data.loc[4107:4111, ['hhs_geo_region', 'region_oxchjgsf', 'region_bhuqouqj', 'region_qufhixun', 'region_lrircsnp', 'region_atmpeygn', 'region_lzgpxyit',
 'region_fpwskwrf', 'region_mlyzmhmf', 'region_dqpwygqj', 'region_kbazzjca']]
column_list = list(data)
pattern = re.compile(r'^region_(?!.*_nan$).*')
column_list =  list(filter(pattern.match, column_list))
for x in column_list: 
    if x not in feature_list: feature_list.append(x)
print (column_list) 

data = data.drop(columns=['msa_Non-MSA', 'msa_MSA, Not Principle  City', 'msa_MSA, Principle City'], errors='ignore')

data = pd.concat([data, pd.get_dummies(data.census_msa, prefix='msa')], axis=1)

 

data.loc[4107:4111, ['census_msa', 'msa_Non-MSA', 'msa_MSA, Not Principle  City', 'msa_MSA, Principle City']]
column_list = list(data)
pattern = re.compile(r'^msa_(?!.*_nan$).*')
column_list =  list(filter(pattern.match, column_list))
for x in column_list: 
    if x not in feature_list: feature_list.append(x)
print (column_list) 

data = data.drop(columns=['pxcmvdjn', 'rucpziij', 'wxleyezf', 'saaquncn', 'xicduogh', 'ldnlellj',
 'wlfvacwt', 'nduyfdeo', 'fcxhlnwr', 'vjjrobsf', 'arjwrbjb', 'atmlpfrs',
 'msuufmds', 'xqicxuve', 'phxvnwax', 'dotnnunm', 'mfikgejo', 'cfqqtusy',
 'mcubkhph', 'haxffmxo', 'qnlwzans'], errors='ignore')

data = pd.concat([data, pd.get_dummies(data.employment_industry, prefix='industry')], axis=1)
 

data.loc[4107:4111, ['employment_industry', 'industry_pxcmvdjn', 'industry_rucpziij', 'industry_wxleyezf', 'industry_saaquncn', 'industry_xicduogh', 'industry_ldnlellj',
 'industry_wlfvacwt', 'industry_nduyfdeo', 'industry_fcxhlnwr', 'industry_vjjrobsf', 'industry_arjwrbjb', 'industry_atmlpfrs',
 'industry_msuufmds', 'industry_xqicxuve', 'industry_phxvnwax', 'industry_dotnnunm', 'industry_mfikgejo', 'industry_cfqqtusy',
 'industry_mcubkhph', 'industry_haxffmxo', 'industry_qnlwzans']]
column_list = list(data)
pattern = re.compile(r'^industry_(?!.*_nan$).*')
column_list =  list(filter(pattern.match, column_list))
for x in column_list: 
    if x not in feature_list: feature_list.append(x)
print (column_list) 

data = data.drop(columns=['xgwztkwe', 'xtkaffoo', 'emcorrxb', 'vlluhbov', 'xqwwgdyp', 'ccgxvspp',
 'qxajmpny', 'kldqjyjy', 'mxkfnird', 'hfxkjkmi', 'bxpfxfdn', 'ukymxvdu',
 'cmhcxjea', 'haliazsg', 'dlvbwzss', 'xzmlyyjv', 'oijqvulv', 'rcertsgn',
 'tfqavkke', 'hodpvpew', 'uqqtjvyb', 'pvmttkik', 'dcjcmpih'], errors='ignore')

data = pd.concat([data, pd.get_dummies(data.employment_occupation, prefix='occupation')], axis=1)

 

data.loc[4107:4111, ['employment_occupation', 'occupation_xgwztkwe', 'occupation_xtkaffoo', 'occupation_emcorrxb', 'occupation_vlluhbov', 'occupation_xqwwgdyp', 'occupation_ccgxvspp',
 'occupation_qxajmpny', 'occupation_kldqjyjy', 'occupation_mxkfnird', 'occupation_hfxkjkmi', 'occupation_bxpfxfdn', 'occupation_ukymxvdu',
 'occupation_cmhcxjea', 'occupation_haliazsg', 'occupation_dlvbwzss', 'occupation_xzmlyyjv', 'occupation_oijqvulv', 'occupation_rcertsgn',
 'occupation_tfqavkke', 'occupation_hodpvpew', 'occupation_uqqtjvyb', 'occupation_pvmttkik', 'occupation_dcjcmpih']]
column_list = list(data)
pattern = re.compile(r'^occupation_(?!.*_nan$).*')
column_list =  list(filter(pattern.match, column_list))
for x in column_list: 
    if x not in feature_list: feature_list.append(x)
print (column_list) 

# Changed List to Numeric_List
# Do we need to do somehting with this for the test data frame as well?

numeric_lists = ['h1n1_concern','h1n1_knowledge','behavioral_antiviral_meds','behavioral_avoidance',
         'behavioral_face_mask','behavioral_wash_hands','behavioral_large_gatherings','behavioral_outside_home',
         'behavioral_touch_face','doctor_recc_h1n1','doctor_recc_seasonal','chronic_med_condition','child_under_6_months',
         'health_worker','health_insurance','opinion_h1n1_vacc_effective','opinion_h1n1_risk','opinion_h1n1_sick_from_vacc',
         'opinion_seas_vacc_effective','opinion_seas_risk','opinion_seas_sick_from_vacc','household_adults','household_children'] 
for x in numeric_lists: 
    if x not in feature_list: feature_list.append(x)
print (feature_list)


features = data.get(feature_list)

print (features.count())

mean_features = data.dropna().mean(numeric_only=True)

imputed_features = data.fillna(mean_features)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):

    print(imputed_features.count())


rf = RandomForestClassifier (n_estimators=300)

rf_scores = cross_val_score(rf, imputed_features.get(feature_list), labels['h1n1_vaccine'], cv=10, n_jobs=4, scoring= 'roc_auc')

print('h1n1', rf_scores.min(), rf_scores.mean(), rf_scores.max())

rf_scores = cross_val_score(rf, imputed_features.get(feature_list), labels['seasonal_vaccine'], cv=10, n_jobs=4, scoring= 'roc_auc')

print('seas', rf_scores.min(), rf_scores.mean(), rf_scores.max())

gb = GradientBoostingClassifier (n_estimators=300)

gb_scores = cross_val_score(gb, imputed_features.get(feature_list), labels['h1n1_vaccine'], cv=10, n_jobs=4, scoring= 'roc_auc')

print('h1n1', gb_scores.min(), gb_scores.mean(), gb_scores.max())

gb_scores = cross_val_score(gb, imputed_features.get(feature_list), labels['seasonal_vaccine'], cv=10, n_jobs=4, scoring= 'roc_auc')

print('seas', gb_scores.min(), gb_scores.mean(), gb_scores.max())

le_age_group = CustomOrderLabelEncoder()

le_age_group.fit(['18 - 34 Years', '35 - 44 Years', '45 - 54 Years', '55 - 64 Years', '65+ Years'])

 

test['enc_age_group'] = le_age_group.transform(test['age_group'])
if 'enc_age_group' not in feature_list:
    feature_list.append('enc_age_group')

 

test.loc[1:10, ['enc_age_group', 'age_group']]

le_education = CustomOrderLabelEncoder()

le_education.fit(['< 12 Years', '12 Years', 'Some College', 'College Graduate'])

 

test['enc_education'] = pd.Series(

    le_education.transform(test['education'][test['education'].notnull()]),

    index=test['education'][test['education'].notnull()].index

)
if 'enc_education' not in feature_list:
    feature_list.append('enc_education')

 

test.loc[1:10, ['enc_education', 'education']]

le_income_poverty = CustomOrderLabelEncoder()

le_income_poverty.fit(['Below Poverty', '<= $75,000, Above Poverty', '> $75,000'])

 

test['enc_income_poverty'] = pd.Series(
    
    le_income_poverty.transform(test['income_poverty'][test['income_poverty'].notnull()]),
    
    index=test['income_poverty'][test['income_poverty'].notnull()].index
               
)
if 'enc_income_poverty' not in feature_list:
    feature_list.append('enc_income_poverty')
 

test.loc[1:10, ['enc_income_poverty', 'income_poverty']]

le_sex = CustomOrderLabelEncoder()

le_sex.fit(['Female', 'Male'])

 

test['enc_sex'] = pd.Series(
    
    le_sex.transform(test['sex'][test['sex'].notnull()]),
    
    index=test['sex'][test['sex'].notnull()].index
               
)
if 'enc_sex' not in feature_list:
    feature_list.append('enc_sex')
 

test.loc[1:10, ['enc_sex', 'sex']]

le_marital_status = CustomOrderLabelEncoder()

le_marital_status.fit(['Not Married', 'Married'])

 

test['enc_marital_status'] = pd.Series(
    
    le_marital_status.transform(test['marital_status'][test['marital_status'].notnull()]),
    
    index=test['marital_status'][test['marital_status'].notnull()].index
               
)
if 'enc_marital_status' not in feature_list:
    feature_list.append('enc_marital_status')
 

test.loc[1:10, ['enc_marital_status', 'marital_status']]

le_rent_or_own = CustomOrderLabelEncoder()

le_rent_or_own.fit(['Own', 'Rent'])

 

test['enc_rent_or_own'] = pd.Series(
    
    le_rent_or_own.transform(test['rent_or_own'][test['rent_or_own'].notnull()]),
    
    index=test['rent_or_own'][test['rent_or_own'].notnull()].index
               
)
if 'enc_rent_or_own' not in feature_list:
    feature_list.append('enc_rent_or_own')
 

test.loc[1:10, ['enc_rent_or_own', 'rent_or_own']]

le_employment_status = CustomOrderLabelEncoder()

le_employment_status.fit(['Not in Labor Force', 'Employed', 'Unemployed'])

 

test['enc_employment_status'] = pd.Series(
    
    le_employment_status.transform(test['employment_status'][test['employment_status'].notnull()]),
    
    index=test['employment_status'][test['employment_status'].notnull()].index
               
)
if 'enc_employment_staus' not in feature_list:
    feature_list.append('enc_employment_status')
 

test.loc[1:10, ['enc_employment_status', 'employment_status']]

test = test.drop(columns=['race_White', 'race_Black', 'race_Other or Multiple', 'race_Hispanic'], errors='ignore')

test = pd.concat([test, pd.get_dummies(test.race, prefix='race')], axis=1)

 

test.loc[4107:4111, ['race', 'race_White', 'race_Black', 'race_Other or Multiple', 'race_Hispanic']]
column_list = list(test)
pattern = re.compile(r'^race_(?!.*_nan$).*')
column_list =  list(filter(pattern.match, column_list))
for x in column_list: 
    if x not in feature_list: feature_list.append(x)
print (column_list) 

test = test.drop(columns=['region_oxchjgsf', 'region_bhuqouqj', 'region_qufhixun', 'region_lrircsnp', 'region_atmpeygn', 'region_lzgpxyit',
 'region_fpwskwrf', 'region_mlyzmhmf', 'region_dqpwygqj', 'region_kbazzjca'], errors='ignore')

test = pd.concat([test, pd.get_dummies(test.hhs_geo_region, prefix='region')], axis=1)

 

test.loc[4107:4111, ['hhs_geo_region', 'region_oxchjgsf', 'region_bhuqouqj', 'region_qufhixun', 'region_lrircsnp', 'region_atmpeygn', 'region_lzgpxyit',
 'region_fpwskwrf', 'region_mlyzmhmf', 'region_dqpwygqj', 'region_kbazzjca']]
column_list = list(test)
pattern = re.compile(r'^region_(?!.*_nan$).*')
column_list =  list(filter(pattern.match, column_list))
for x in column_list: 
    if x not in feature_list: feature_list.append(x)
print (column_list) 

test = test.drop(columns=['msa_Non-MSA', 'msa_MSA, Not Principle  City', 'msa_MSA, Principle City'], errors='ignore')

test = pd.concat([test, pd.get_dummies(data.census_msa, prefix='msa')], axis=1)

 

test.loc[4107:4111, ['census_msa', 'msa_Non-MSA', 'msa_MSA, Not Principle  City', 'msa_MSA, Principle City']]
column_list = list(test)
pattern = re.compile(r'^msa_(?!.*_nan$).*')
column_list =  list(filter(pattern.match, column_list))
for x in column_list: 
    if x not in feature_list: feature_list.append(x)
print (column_list) 

test = test.drop(columns=['pxcmvdjn', 'rucpziij', 'wxleyezf', 'saaquncn', 'xicduogh', 'ldnlellj',
 'wlfvacwt', 'nduyfdeo', 'fcxhlnwr', 'vjjrobsf', 'arjwrbjb', 'atmlpfrs',
 'msuufmds', 'xqicxuve', 'phxvnwax', 'dotnnunm', 'mfikgejo', 'cfqqtusy',
 'mcubkhph', 'haxffmxo', 'qnlwzans'], errors='ignore')

test = pd.concat([test, pd.get_dummies(test.employment_industry, prefix='industry')], axis=1)
 

test.loc[4107:4111, ['employment_industry', 'industry_pxcmvdjn', 'industry_rucpziij', 'industry_wxleyezf', 'industry_saaquncn', 'industry_xicduogh', 'industry_ldnlellj',
 'industry_wlfvacwt', 'industry_nduyfdeo', 'industry_fcxhlnwr', 'industry_vjjrobsf', 'industry_arjwrbjb', 'industry_atmlpfrs',
 'industry_msuufmds', 'industry_xqicxuve', 'industry_phxvnwax', 'industry_dotnnunm', 'industry_mfikgejo', 'industry_cfqqtusy',
 'industry_mcubkhph', 'industry_haxffmxo', 'industry_qnlwzans']]
column_list = list(test)
pattern = re.compile(r'^industry_(?!.*_nan$).*')
column_list =  list(filter(pattern.match, column_list))
for x in column_list: 
    if x not in feature_list: feature_list.append(x)
print (column_list) 

test = test.drop(columns=['xgwztkwe', 'xtkaffoo', 'emcorrxb', 'vlluhbov', 'xqwwgdyp', 'ccgxvspp',
 'qxajmpny', 'kldqjyjy', 'mxkfnird', 'hfxkjkmi', 'bxpfxfdn', 'ukymxvdu',
 'cmhcxjea', 'haliazsg', 'dlvbwzss', 'xzmlyyjv', 'oijqvulv', 'rcertsgn',
 'tfqavkke', 'hodpvpew', 'uqqtjvyb', 'pvmttkik', 'dcjcmpih'], errors='ignore')

test = pd.concat([test, pd.get_dummies(test.employment_occupation, prefix='occupation')], axis=1)

 

test.loc[4107:4111, ['employment_occupation', 'occupation_xgwztkwe', 'occupation_xtkaffoo', 'occupation_emcorrxb', 'occupation_vlluhbov', 'occupation_xqwwgdyp', 'occupation_ccgxvspp',
 'occupation_qxajmpny', 'occupation_kldqjyjy', 'occupation_mxkfnird', 'occupation_hfxkjkmi', 'occupation_bxpfxfdn', 'occupation_ukymxvdu',
 'occupation_cmhcxjea', 'occupation_haliazsg', 'occupation_dlvbwzss', 'occupation_xzmlyyjv', 'occupation_oijqvulv', 'occupation_rcertsgn',
 'occupation_tfqavkke', 'occupation_hodpvpew', 'occupation_uqqtjvyb', 'occupation_pvmttkik', 'occupation_dcjcmpih']]
column_list = list(test)
pattern = re.compile(r'^occupation_(?!.*_nan$).*')
column_list =  list(filter(pattern.match, column_list))
for x in column_list: 
    if x not in feature_list: feature_list.append(x)
print (column_list) 

print(feature_list)

features = test.get(feature_list)

mean_features = test.dropna().mean(numeric_only=True)

test_imputed_features = test.fillna(mean_features)

print (test_imputed_features.count())





h1n1_gb = GradientBoostingClassifier(

  n_estimators=300,

)

seas_gb = GradientBoostingClassifier(

  n_estimators=300,

)

h1n1_rf = RandomForestClassifier(

  n_estimators=300,

)

seas_rf = RandomForestClassifier(

  n_estimators=300,

)

%%time
h1n1_gb.fit(imputed_features.get(feature_list), labels['h1n1_vaccine'])

%%time
seas_gb.fit(imputed_features.get(feature_list), labels['seasonal_vaccine'])

print (labels) 

 h1n1_yhat = h1n1_gb.predict_proba(test_imputed_features.get(feature_list))

seas_yhat = seas_gb.predict_proba(test_imputed_features.get(feature_list))

print (seas_yhat)

submission_out = pd.DataFrame(list(zip(h1n1_yhat[:, 1], seas_yhat[:, 1])), test_imputed_features['respondent_id'].values, columns=['h1n1_vaccine','seasonal_vaccine'])


print (submission_out)

submission_out.index.name = 'respondent_id'

submission_out.to_csv(os.path.abspath("") + '/submission-'+t.strftime('%b-%d-%Y_%H%M',t.localtime())+'.csv')

